In [2]:
import json
from pymorphy2 import MorphAnalyzer
from tqdm import tqdm
import pandas as pd
import numpy as np
import random
import nltk
from pymystem3 import Mystem

In [ ]:
!ls data

In [ ]:
!unzip ...

In [3]:
with open('data/corpus.json') as f:
    corpus = json.load(f)

In [4]:
'Текстов - {}, слов - {}'.format(len(corpus), sum([len(sample) for sample in corpus]))

'Текстов - 1157366, слов - 16028874'

In [ ]:
morph = MorphAnalyzer()

In [5]:
m = Mystem()

In [8]:
def get_lemma(word):
    
    word_data = m.lemmatize(word)[0]
    
    return word_data

## Соберем словарь лем
В нашем корпусе 16028874 слов. Лемматизировать весь корпус будет очень долго. Давайте лучше соберем словарь уникальных слов и будет лемматизировать только уникальные слова.

In [9]:
tok2lemma = {}

for text in tqdm(corpus):
    for tok in text:
        if tok not in tok2lemma:
            tok2lemma[tok] = get_lemma(tok)

100%|██████████| 1157366/1157366 [03:11<00:00, 6054.75it/s] 


In [10]:
# ключ - уникальное слово в нашем корпусе, значение - его лемма
tok2lemma['уехали']

'уезжать'

In [11]:
len(tok2lemma)

193435

In [12]:
stopwords = nltk.corpus.stopwords.words('russian')

# Замена слов леммами
Так как теперь к каждому слову из нашего корпуса мы знаем лемму, то давайте каждое слово заменим на его лемму и уберем стоп слова.
Это работает гораздо(!) быстрее, чем если бы мы в корпусе для каждого слова каждый раз рассчитывали лемму (с помощью пайморфи), 
потому что теперь нам надо вызвать пайморфи 193435 раз вместо 16028874.

In [13]:
lemmas_corpus = [[tok2lemma[tok] for tok in text if tok not in stopwords and tok]
                 for text in tqdm(corpus)]

100%|██████████| 1157366/1157366 [00:42<00:00, 27265.36it/s]


# Соберем частотный словарь

In [14]:
freq = {}

for text in tqdm(lemmas_corpus):
    for tok in text:
        if tok in freq:
            freq[tok] += 1
        else:
            freq[tok] = 1

100%|██████████| 1157366/1157366 [00:04<00:00, 265206.85it/s]


In [15]:
freq_df = pd.DataFrame(data={'word': list(freq.keys()), 'n_entries': list(freq.values())})

In [16]:
freq_df.sort_values(by=['n_entries'], ascending=False, inplace=True)

In [17]:
freq_df.head()

,word,n_entries
21,#,413016
25,банк,187296
47,карта,156216
3,банка,146538
132,кредит,86866


In [18]:
# уникальных слов в словаре
freq_df.shape

(74882, 2)

In [19]:
freq_df.tail()

,word,n_entries
43925,надоедливо,1
43926,воспитательница,1
43928,неподозревающий,1
43929,признки,1
74881,подпал,1


In [20]:
n_words = freq_df.n_entries.sum()

# Замена редких слов
В нашем корпусе осталось много слов, которые встречаются очень редко. Давайте мы редкие слова заменим на специальный токе UNK - unknown. Так мы разительно сократим размер нашего словаря слов с незначительной потерей информации.

In [21]:
print('Доля слов, которые мы заменим на UNK:')

for threshold in np.arange(5, 36, 5):
    
    sub_df = freq_df[freq_df.n_entries < threshold]
    
    unk_freq = sub_df['n_entries'].sum() * 100 / n_words
    
    print('Порог отсечения - {}, доля UNK - {:.2f} %, слов в слове - {}, удалили - {} слов'.format(
        threshold, unk_freq, freq_df.shape[0] - sub_df.shape[0], sub_df.shape[0]))

Доля слов, которые мы заменим на UNK:
Порог отсечения - 5, доля UNK - 0.74 %, слов в слове - 22271, удалили - 52611 слов
Порог отсечения - 10, доля UNK - 1.14 %, слов в слове - 15957, удалили - 58925 слов
Порог отсечения - 15, доля UNK - 1.46 %, слов в слове - 13099, удалили - 61783 слов
Порог отсечения - 20, доля UNK - 1.75 %, слов в слове - 11351, удалили - 63531 слов
Порог отсечения - 25, доля UNK - 1.99 %, слов в слове - 10201, удалили - 64681 слов
Порог отсечения - 30, доля UNK - 2.21 %, слов в слове - 9352, удалили - 65530 слов
Порог отсечения - 35, доля UNK - 2.42 %, слов в слове - 8667, удалили - 66215 слов


In [22]:
# кажется, что оптимально, но обычно берут меньше
threshold = 15

In [23]:
vocab = freq_df[freq_df.n_entries >= threshold]

In [24]:
words = set(vocab.word)

In [25]:
len(words)

13099

In [26]:
'Мы сократили наш словарь в {:.2f} раз с потерей 1.51 % всех слов'.format(freq_df.shape[0] / len(words))

'Мы сократили наш словарь в 5.72 раз с потерей 1.51 % всех слов'

In [27]:
def get_correct_words(word):
    
    if word in words:
        return word
    else:
        return 'UNK'

In [28]:
# заменим слово токеном UNK, если его нет в нашем новом словаре
processed_corpus = [[get_correct_words(tok) for tok in text] for text in tqdm(lemmas_corpus)]

100%|██████████| 1157366/1157366 [00:07<00:00, 163092.59it/s]


In [29]:
def drop_duplicate_unks(tokens):
    
    output_tokens = []
    
    for tok in tokens:
        
        if tok == 'UNK' and output_tokens and output_tokens[-1] == 'UNK':
            continue
            
        output_tokens.append(tok)
            
    return output_tokens

In [30]:
sample_text = 'думать далее милый барышня UNK UNK тинькоф звонить неделя'.split()

In [31]:
sample_text

['думать',
 'далее',
 'милый',
 'барышня',
 'UNK',
 'UNK',
 'тинькоф',
 'звонить',
 'неделя']

In [32]:
drop_duplicate_unks(sample_text)

['думать', 'далее', 'милый', 'барышня', 'UNK', 'тинькоф', 'звонить', 'неделя']

In [33]:
# дедублируем подряд идущие унки (оставим только один)
processed_corpus = [drop_duplicate_unks(sample) for sample in tqdm(processed_corpus)]

100%|██████████| 1157366/1157366 [00:06<00:00, 192670.29it/s]


In [34]:
texts_with_unk = [text for text in processed_corpus if 'UNK' in text]
'Текстов с унками - {:.2f} %'.format(len(texts_with_unk) * 100 / len(processed_corpus))

'Текстов с унками - 10.90 %'

In [35]:
# посмотрим на тексты с унками
for text in random.sample(texts_with_unk, k=5):
    print(' '.join(text))

должный претензия ко компания ооо UNK
свой слово UNK подтвердить принимать должный действовать клиент
наоборот хотеть лишаться доступ деньги UNK дебетовый карта втб #
оформлять UNK предложение кредит наличные сотрудник понадобиться # час
звать UNK номер телефон # постоянно поступать звонок ваш банка


In [36]:
# выглядит не так плохо

In [37]:
random.shuffle(processed_corpus)

# Выберем подвыборку данных
Чтобы быстрее выучить word2vec

In [42]:
sub_data = processed_corpus[:500000]

In [43]:
with open('data/processed_corpus_4.json', 'w') as f:
    json.dump(sub_data, f, ensure_ascii=False)